In [1]:
import sys
sys.path.append("/mnt/home/TF_NEW/tf-transformers/src/")

In [3]:
from transformers import TFMT5Model, T5Tokenizer
from tf_transformers.models import mT5Model
import tensorflow as tf
import json
import os

from tf_transformers.utils import convert_mt5_hf_to_tf_transformers


In [44]:
# Load HF model

# Always do this
tf.keras.backend.clear_session()

model_hf_location = '/Users/PRVATE/HUggingFace_Models/mt5-small/'
model_hf = TFMT5Model.from_pretrained(model_hf_location)
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")


Some layers from the model checkpoint at /Users/PRVATE/HUggingFace_Models/mt5-small/ were not used when initializing TFMT5Model: ['lm_head']
- This IS expected if you are initializing TFMT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFMT5Model were initialized from the model checkpoint at /Users/PRVATE/HUggingFace_Models/mt5-small/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5Model for predictions without further training.


In [45]:
model_layer, model, config = mT5Model(model_name='mt5-small')

INFO:absl:Initialized Variables


INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)


In [49]:
convert_mt5_hf_to_tf_transformers(model_hf, model, config['encoder'])

INFO:absl:Deleteing huggingface model for saving memory
INFO:absl:Done assigning ENCODER variables weights 75
INFO:absl:Deleteing huggingface model for saving memory
INFO:absl:Done assigning DECODER variables weights 115


In [ ]:
### Save checkpoint
model.save_checkpoint("/Users/PRVATE/LegacyAI_models/mt5-small/", overwrite=True)

In [50]:
inputs_sample = {'encoder_input_ids': tf.constant([[8774,    6,   82, 1782,   19, 5295]]), 
                 'encoder_input_mask': tf.constant([[1, 1, 1, 1, 1, 1]]), 
                 'decoder_input_ids': tf.constant([[8774,    6,   82, 1782,   19, 5295]])}

res = model(inputs_sample)
for k, v in res.items():
    print(k, tf.reduce_sum(v))
    
# Reference 

# token_embeddings tf.Tensor(-125.860725, shape=(), dtype=float32)
# all_layer_token_embeddings tf.Tensor(-192280.42, shape=(), dtype=float32)
# token_logits tf.Tensor(-103934104.0, shape=(), dtype=float32)
# last_token_logits tf.Tensor(-19607434.0, shape=(), dtype=float32)

token_embeddings tf.Tensor(-20.499813, shape=(), dtype=float32)
all_layer_token_embeddings tf.Tensor(-222329.97, shape=(), dtype=float32)
token_logits tf.Tensor(42779436.0, shape=(), dtype=float32)
last_token_logits tf.Tensor(5198241.5, shape=(), dtype=float32)
encoder_hidden_states tf.Tensor(52.110146, shape=(), dtype=float32)


In [51]:
res = model_hf(input_ids=inputs_sample['encoder_input_ids'], decoder_input_ids=inputs_sample['decoder_input_ids'])
# for k, v in res.items():
#     print(k, tf.reduce_sum(v))
    

In [52]:
tf.reduce_sum(res.last_hidden_state)

<tf.Tensor: shape=(), dtype=float32, numpy=-20.501228>

In [54]:
# Lets do a simple decode

# Greedy decoding ( Train mode)

# -3.5792198 2739 tf.Tensor(-12332620.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739]], shape=(1, 2), dtype=int32)
# -----------------
# 39.560272 4445 tf.Tensor(-14821873.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445]], shape=(1, 3), dtype=int32)
# -----------------
# -31.229576 436 tf.Tensor(-14894206.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436]], shape=(1, 4), dtype=int32)
# -----------------
# -67.342186 292 tf.Tensor(-16596788.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292]], shape=(1, 5), dtype=int32)
# -----------------
# -6.252556 58 tf.Tensor(-15607945.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58]], shape=(1, 6), dtype=int32)
# -----------------
# -118.80247 1 tf.Tensor(-18228060.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58    1]], shape=(1, 7), dtype=int32)
# -----------------
# -63.030956 1 tf.Tensor(-12004529.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58    1    1]], shape=(1, 8), dtype=int32)
# -----------------
# -16.752626 10 tf.Tensor(-9946732.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58    1    1   10]], shape=(1, 9), dtype=int32)
# -----------------
# -31.18003 3 tf.Tensor(-10146046.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58    1    1   10    3]], shape=(1, 10), dtype=int32)
# -----------------
# 1.3379288 2 tf.Tensor(-11181468.0, shape=(), dtype=float32)
# tf.Tensor([[   0 2739 4445  436  292   58    1    1   10    3    2]], shape=(1, 11), dtype=int32)

encoder_input_ids   = tf.constant([[37194, 5413, 288, 20567, 267, 5100, 6801, 418, 521, 291, 1]])
encoder_input_mask  = tf.ones_like(encoder_input_ids)
decoder_input_ids   = tf.constant([[0]])
encoder_hidden_states = tf.zeros((1, 11, 512))

encoder_decoder_inputs = {'encoder_input_ids': encoder_input_ids, 
                          'encoder_input_mask': encoder_input_mask, 
                          'decoder_input_ids': decoder_input_ids
                         }
best_res = []

for i in range(10):


    results = model(encoder_decoder_inputs)
    best_prob, best_index = tf.nn.top_k(results['last_token_logits'][0], k=1)
    # Concataning with previous
    new_dec_ids = tf.concat([encoder_decoder_inputs['decoder_input_ids'], [best_index]], axis=1)
    encoder_decoder_inputs['decoder_input_ids'] = new_dec_ids
    print(best_prob[0].numpy() , best_index[0].numpy(), tf.reduce_sum(results['last_token_logits']))
    best_res.append((best_prob[0].numpy(), best_index[0].numpy()))
    print(new_dec_ids)
    print('-----------------')

3216.4043 217189 tf.Tensor(16623835.0, shape=(), dtype=float32)
tf.Tensor([[     0 217189]], shape=(1, 2), dtype=int32)
-----------------
5409.611 217189 tf.Tensor(32614544.0, shape=(), dtype=float32)
tf.Tensor([[     0 217189 217189]], shape=(1, 3), dtype=int32)
-----------------
2621.8142 217189 tf.Tensor(11411722.0, shape=(), dtype=float32)
tf.Tensor([[     0 217189 217189 217189]], shape=(1, 4), dtype=int32)
-----------------
1972.4235 147087 tf.Tensor(4441350.5, shape=(), dtype=float32)
tf.Tensor([[     0 217189 217189 217189 147087]], shape=(1, 5), dtype=int32)
-----------------
2357.5967 14607 tf.Tensor(27313216.0, shape=(), dtype=float32)
tf.Tensor([[     0 217189 217189 217189 147087  14607]], shape=(1, 6), dtype=int32)
-----------------
2490.151 83261 tf.Tensor(25318396.0, shape=(), dtype=float32)
tf.Tensor([[     0 217189 217189 217189 147087  14607  83261]], shape=(1, 7), dtype=int32)
-----------------
2035.7072 219777 tf.Tensor(13057432.0, shape=(), dtype=float32)
tf.Tenso

In [55]:
model_layer , model, config = mT5Model(model_name='mt5-small',
                                      is_training=False,
                                      pipeline_mode='auto-regressive')

INFO:absl:We are overwriding `is_training` is False to `is_training` to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Initialized Variables
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_input_ids ---> Tensor("decoder_input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:decoder_all_cache_key ---> Tensor("all_cache_key:0", shape=(None, None, 6, None, 64), dtype=float32)
INFO:absl:decoder_all_cache_value ---> Tensor("all_cache_value:0", shape=(None, None, 6, None, 64), dtype=float32)
INFO:absl:encoder_hidden_states ---> Tensor("encoder_hidden_states:0", shape=(None, None, 512), dtype=float32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:encoder_input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:encoder_i

In [56]:
model.load_checkpoint("/Users/PRVATE/LegacyAI_models/mt5-small/")


Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.EncoderDecoder object at 0x169fdb4c0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x15e9fa220>).



Two checkpoint references resolved to different objects (<tf_transformers.models.encoder_decoder.EncoderDecoder object at 0x169fdb4c0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x15e9fa220>).
INFO:absl:Succesful: Model checkpoints matched


In [57]:
# Greedy decoding Using (AR pipeline caching)

encoder_input_ids   = tf.constant([[37194, 5413, 288, 20567, 267, 5100, 6801, 418, 521, 291, 1]])
encoder_input_mask  = tf.ones_like(encoder_input_ids)
decoder_input_ids   = tf.constant([[0]])
encoder_hidden_states = tf.zeros((1, 11, config['encoder']['embedding_size']))


batch_size = 1
sequence_length = 1
# (self.num_hidden_layers, batch_size, self.num_attention_heads, sequence_length, self.embedding_size//self.num_attention_heads)
all_cache_key  =  tf.zeros((config['encoder']['num_hidden_layers'],
                            batch_size, 
                            config['encoder']['num_attention_heads'], 
                            sequence_length, config['encoder']['embedding_size']//config['encoder']['num_attention_heads']))


all_cache_value = tf.zeros_like(all_cache_key)

encoder_decoder_inputs = {'encoder_input_ids': encoder_input_ids, 
                          'encoder_input_mask': encoder_input_mask, 
                          'decoder_input_ids': decoder_input_ids,
                          'encoder_hidden_states': encoder_hidden_states,
                          'decoder_all_cache_key': all_cache_key,
                          'decoder_all_cache_value': all_cache_value
                         }

best_res = []
for i in range(10):


    results = model(encoder_decoder_inputs)
    best_prob, best_index = tf.nn.top_k(results['last_token_logits'][0], k=1)
    
    # Concataning with previous
    new_dec_ids = tf.expand_dims(best_index, axis=1)
    encoder_decoder_inputs['decoder_input_ids'] = new_dec_ids
    encoder_decoder_inputs['decoder_all_cache_key'] = results['decoder_all_cache_key']
    encoder_decoder_inputs['decoder_all_cache_value'] = results['decoder_all_cache_value']
    encoder_decoder_inputs['encoder_hidden_states'] = results['encoder_hidden_states']
    print(best_prob[0].numpy() , best_index[0].numpy())
    best_res.append((best_prob[0].numpy() , best_index[0].numpy()))
    print(new_dec_ids)
    print('-----------------')
    

142.14633 217189
tf.Tensor([[217189]], shape=(1, 1), dtype=int32)
-----------------
239.07336 217189
tf.Tensor([[217189]], shape=(1, 1), dtype=int32)
-----------------
115.86856 217189
tf.Tensor([[217189]], shape=(1, 1), dtype=int32)
-----------------
87.16968 147087
tf.Tensor([[147087]], shape=(1, 1), dtype=int32)
-----------------
104.192024 14607
tf.Tensor([[14607]], shape=(1, 1), dtype=int32)
-----------------
110.05008 83261
tf.Tensor([[83261]], shape=(1, 1), dtype=int32)
-----------------
89.96649 219777
tf.Tensor([[219777]], shape=(1, 1), dtype=int32)
-----------------
112.242065 138182
tf.Tensor([[138182]], shape=(1, 1), dtype=int32)
-----------------
124.41264 138182
tf.Tensor([[138182]], shape=(1, 1), dtype=int32)
-----------------
147.09375 207669
tf.Tensor([[207669]], shape=(1, 1), dtype=int32)
-----------------
